<a href="https://colab.research.google.com/github/ThallesLM/spaceX_data_analysis/blob/main/labs_jupyter_spacex_Data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


 ## Lab 2: Data wrangling


Estimated time needed: **60** minutes


Neste laboratório, realizaremos algumas análises exploratórias de dados (EDA) para encontrar alguns padrões nos dados e determinar qual seria o rótulo para treinar modelos supervisionados.

No conjunto de dados, existem vários casos diferentes em que o propulsor não pousou com sucesso. Às vezes, tentava-se um pouso, mas falhava devido a um acidente; por exemplo, <code>True Ocean</code> significa que o resultado da missão foi desembarcado com sucesso em uma região específica do oceano, enquanto <code>False Ocean</code> significa que o resultado da missão foi desembarcado sem sucesso em uma região específica do oceano . <code>RTLS verdadeiro</code> significa que o resultado da missão foi pousado com sucesso em uma plataforma terrestre. <code>RTLS falso</code> significa que o resultado da missão foi pousado sem sucesso em uma plataforma terrestre.<code>ASDS verdadeiro</code> significa que o resultado da missão foi pousado com sucesso em um navio drone. <code>Falso ASDS</code> significa que o resultado da missão foi pousado sem sucesso em um navio drone.

Neste laboratório, converteremos principalmente esses resultados em rótulos de treinamento com `1` significa que o booster acertou com sucesso e `0` significa que não teve sucesso.

Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## Objectives
Perform exploratory  Data Analysis and determine Training Labels

- Exploratory Data Analysis
- Determine Training Labels


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries.


In [1]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

### Data Analysis


Load Space X dataset, from last section.


In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identify and calculate the percentage of the missing values in each attribute


In [3]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Identify which columns are numerical and categorical:


In [4]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### TAREFA 1: Calcular o número de lançamentos em cada site

Os dados contêm várias instalações de lançamento do Space X: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>Cape Canaveral Space</a> Launch Complex 40 <b>VAFB SLC 4E </b> , Complexo de lançamento espacial 4E da Base Aérea de Vandenberg <b>(SLC-4E)</b>, Complexo de lançamento 39A do Centro Espacial Kennedy <b>KSC LC 39A </b>. A localização de cada lançamento é colocada na coluna <code >LaunchSite</code>


A seguir, vamos ver a quantidade de lançamentos de cada site.

Use o método <code>value_counts()</code> na coluna <code>LaunchSite</code> para determinar o número de lançamentos em cada site:


In [6]:
# Apply value_counts() on column LaunchSite
df.LaunchSite.value_counts()

LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64

Each launch aims to an dedicated orbit, and here are some common orbit types:




* <b>LEO</b>: Órbita baixa da Terra (LEO) é uma órbita centrada na Terra com uma altitude de 2.000 km (1.200 mi) ou menos (aproximadamente um terço do raio da Terra),[1] ou com pelo menos 11,25 períodos por dia (um período orbital de 128 minutos ou menos) e uma excentricidade inferior a 0,25.[2] A maioria dos objetos artificiais no espaço sideral está em LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Órbitas Terrestres Muito Baixas (VLEO) podem ser definidas como órbitas com altitude média abaixo de 450 km. Operar nessas órbitas pode fornecer uma série de benefícios para a espaçonave de observação da Terra, pois a espaçonave opera mais perto da observação<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.


* <b>GTO</b> Uma órbita geossíncrona é uma órbita alta da Terra que permite que os satélites correspondam à rotação da Terra. Localizada a 35.786 quilómetros acima do equador da Terra, esta posição é um local valioso para monitorização do tempo, comunicações e vigilância. Como o satélite orbita na mesma velocidade que a Terra gira, o satélite parece permanecer no lugar ao longo de uma única longitude, embora possa flutuar de norte a sul”, escreveu a NASA em seu site do Observatório da Terra <a href="https:/ /www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.


* <b>SSO (ou SO)</b>: É uma órbita sincronizada com o Sol, também chamada de órbita heliossíncrona, é uma órbita quase polar ao redor de um planeta, na qual o satélite passa sobre qualquer ponto da superfície do planeta no mesma hora solar média local <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.
    
    
    
* <b>ES-L1 </b>:Nos pontos de Lagrange as forças gravitacionais dos dois grandes corpos se anulam de tal forma que um pequeno objeto colocado em órbita ali fica em equilíbrio em relação ao centro de massa do grande corpos. L1 é um desses pontos entre o sol e a terra <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .
    
    
* <b>HEO</b> Uma órbita altamente elíptica, é uma órbita elíptica com alta excentricidade, geralmente referindo-se a uma órbita ao redor da Terra <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6 ]</a>.


* <b> ISS </b> Uma estação espacial modular (satélite artificial habitável) em órbita baixa da Terra. É um projeto colaborativo multinacional entre cinco agências espaciais participantes: NASA (Estados Unidos), Roscosmos (Rússia), JAXA (Japão), ESA (Europa) e CSA (Canadá)<a href="https://en.wikipedia .org/wiki/International_Space_Station"> [7] </a>


* <b> MEO </b> Órbitas geocêntricas variando em altitude de 2.000 km (1.200 milhas) até logo abaixo da órbita geossíncrona a 35.786 quilômetros (22.236 milhas). Também conhecida como órbita circular intermediária. Estes são "mais comumente a 20.200 quilômetros (12.600 milhas), ou 20.650 quilômetros (12.830 milhas), com um período orbital de 12 horas <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8 ] </a>


* <b> HEO </b> Órbitas geocêntricas acima da altitude da órbita geossíncrona (35.786 km ou 22.236 milhas) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a >


* <b> GEO </b> É uma órbita geossíncrona circular 35.786 quilômetros (22.236 milhas) acima do equador da Terra e seguindo a direção da rotação da Terra <a href="https://en.wikipedia.org/wiki/Geostationary_orbit" > [10] </a>


* <b> PO </b> É um tipo de satélite em que um satélite passa acima ou quase acima de ambos os pólos do corpo que está sendo orbitado (geralmente um planeta como a Terra <a href="https://en. wikipedia.org/wiki/Polar_orbit"> [11] </a>

alguns são mostrados no gráfico a seguir:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### TASK 2: Calculate the number and occurrence of each orbit


 Use o método <code>.value_counts()</code> para determinar o número e a ocorrência de cada órbita na coluna <code>Orbit</code>


In [7]:
# Apply value_counts on Orbit column
df.Orbit.value_counts()

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: count, dtype: int64

### TASK 3: Calculate the number and occurence of mission outcome of the orbits


Use o método <code>.value_counts()</code> na coluna <code>Outcome</code> para determinar o número de <code>landing_outcomes</code>. Em seguida, atribua-o a uma variável landing_outcomes.


In [9]:
landing_outcomes = df['Outcome'].value_counts()

<code>True Ocean</code> significa que o resultado da missão foi pousado com sucesso em uma região específica do oceano, enquanto <code>False Ocean</code> significa que o resultado da missão foi pousado sem sucesso em uma região específica do oceano. <code>RTLS verdadeiro</code> significa que o resultado da missão foi pousado com sucesso em uma plataforma terrestre. <code>RTLS falso</code> significa que o resultado da missão foi pousado sem sucesso em uma plataforma terrestre.<code>ASDS verdadeiro</code> significa que o resultado da missão foi pousado com sucesso em um navio drone. <code>Falso ASDS</code> significa que o resultado da missão foi pousado sem sucesso em um navio drone. <code>None ASDS</code> e <code>None None</code> representam uma falha no pouso.

In [10]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


Criamos um conjunto de resultados onde o segundo estágio não foi bem-sucedido:


In [11]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### TASK 4: Create a landing outcome label from Outcome column


Usando o <code>Outcome</code>, crie uma lista onde o elemento é zero se a linha correspondente em <code>Outcome</code> estiver no conjunto <code>bad_outcome</code>; caso contrário, é um. Em seguida, atribua-o à variável <code>landing_class</code>:

In [13]:
# Supondo que 'bad_outcome' é o conjunto de resultados ruins
landing_class = [0 if outcome in bad_outcomes else 1 for outcome in df['Outcome']]


This variable will represent the classification variable that represents the outcome of each launch. If the value is zero, the  first stage did not land successfully; one means  the first stage landed Successfully


In [14]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [15]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


Podemos usar a seguinte linha de código para determinar a taxa de sucesso:

In [16]:
df["Class"].mean()

0.6666666666666666

In [17]:
df.to_csv("dataset_part_2.csv", index=False)

We can now export it to a CSV for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.


<code>df.to_csv("dataset_part_2.csv", index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla    | Changed Markdown |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef      | updating the input data |
| 2021-05-026       | 1.1.    | Joseph      | updating the input data |


Copyright © 2021 IBM Corporation. All rights reserved.
